In [2]:
import pandas as pd

# import dataframes drom preprocessed news
df_1 = pd.read_csv('preprocessed_news/Crypto_Currency_News_f.csv')
df_2 = pd.read_csv('preprocessed_news/CryptoCurrencies_f.csv')
df_3 = pd.read_csv('preprocessed_news/CryptoCurrency_f.csv')
df_4 = pd.read_csv('preprocessed_news/Cryptomarkets_f.csv')
df_5 = pd.read_csv('preprocessed_news/eth_f.csv')
df_6 = pd.read_csv('preprocessed_news/ethfinance_f.csv')
df_7 = pd.read_csv('preprocessed_news/ethtrader_f.csv')

In [67]:
# We are going to create a dictionary with all the words in the news
# We will use this dictionary to tokenize the news
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# set of stopwords
# keep negation words (for all apostrophe negations, the token will become n't)
keep = {'no', 'not', 'nor', 'don', 'ain', 'aren', 'couldn', 'didn', 'doesn', 
        'hadn', 'hasn', 'haven', 'isn', 'mightn', 'mustn', 'needn', 'shan', 
        'shouldn', 'wasn', 'weren', 'won', 'wouldn', 'more', 'most', 'less'}
stop_words = set(stopwords.words('english'))
extra_sw = {'http', '\'s'}
stop_words = stop_words.union(extra_sw)

# set of punctuation
punct = set(string.punctuation) - {'$', '\''}
punct.add('’')
# extra_p = {'’', '..', '\'\'', '``', '...', '....', "''", '...'}
# punct = punct.union(extra_p)

# lemmatizer, to get the root of the word
lemmatizer = WordNetLemmatizer()

# set of numbers
numbers = set('0123456789')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lcano\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lcano\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lcano\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [74]:
# create dictionary
def create_dictionary(df):
    # dictionary = {word: token}
    dictionary = defaultdict(int)
    token = 1
    # Some important words to add to the dictionary
    important_words = ['sos', 'eos', 'ethereum', 'bitcoin', 'eth', 'btc']
    for word in important_words:
        dictionary[word] = token
        token += 1
    # iterate over the news
    # df.values are text_1 to text_10 of each day
    for news in df.values:
        for text in news:
            # tokenize the text
            tokens = word_tokenize(text)
            for word in tokens:
                word = word.lower()
                # if word starts with ' remove it (to prevent cases example: why and 'why)
                if word[0] == '\'':
                    if len(word) == 1:
                        continue
                    word = word[1:]
                # remove punctuation
                if not any((c in punct) for c in word):
                    # remove numbers
                    if word[0] in numbers:
                        continue
                    # remove stopwords
                    if word in stop_words:
                        continue
                    # lemmatize
                    # do not lemmatize SOS and EOS
                    if word != 'sos' and word != 'eos':
                        word = lemmatizer.lemmatize(word)
                        # is possible that the word root is a stopword
                        if word in stop_words:
                            continue
                    # add to dictionary
                    if dictionary[word] == 0:
                        dictionary[word] = token
                        token += 1
    return dictionary

    # dictionary = defaultdict(int)
    # # Some important words to add to the dictionary
    # important_words = ['sos', 'eos', 'ethereum', 'bitcoin', 'eth', 'btc']
    # for word in important_words:
    #     dictionary[word] = 1
    # for news in df.values: # df.values are text_1 to text_10 of each day
    #     for text in news:
    #         if type(text) == str:
    #             for word in word_tokenize(text):
    #                 word = word.lower()
    #                 # if word starts with ' remove it (to prevent cases example: why and 'why)
    #                 # but skip to next word if the word is only '
    #                 if word[0] == '\'':
    #                     if len(word) == 1:
    #                         continue
    #                     word = word[1:]
    #                 # check that the word does not have any char from special 
    #                 if not any((c in punct) for c in word):
    #                     if word not in stop_words and word not in numbers:
    #                         # do not lemmatize SOS and EOS
    #                         if word != 'sos' and word != 'eos':
    #                             word = lemmatizer.lemmatize(word)
    #                             # is possible that the word root is a stopword
    #                             if word in stop_words:
    #                                 continue
    #                         dictionary[word] += 1
    # return dictionary

# create dictionary
dictionary_1 = create_dictionary(df_1)
# add the rest of df to the dictionary, new words will be added, repeated words will stay the same
dictionary_2 = create_dictionary(df_2)
dictionary_3 = create_dictionary(df_3)
dictionary_4 = create_dictionary(df_4)
dictionary_5 = create_dictionary(df_5)
dictionary_6 = create_dictionary(df_6)
dictionary_7 = create_dictionary(df_7)

# merge dictionaries
dictionary = dictionary_1.copy()
dictionary.update(dictionary_2)
dictionary.update(dictionary_3)
dictionary.update(dictionary_4)
dictionary.update(dictionary_5)
dictionary.update(dictionary_6)
dictionary.update(dictionary_7)


# dictionary = create_dictionary(df_1)
# # add the rest of df to the dictionary, new words will be added, repeated words will be updated by summing the frequency
# for word, freq in create_dictionary(df_2).items():
#     dictionary[word] += freq
# for word, freq in create_dictionary(df_3).items():
#     dictionary[word] += freq
# for word, freq in create_dictionary(df_4).items():
#     dictionary[word] += freq
# for word, freq in create_dictionary(df_5).items():
#     dictionary[word] += freq
# for word, freq in create_dictionary(df_6).items():
#     dictionary[word] += freq
# for word, freq in create_dictionary(df_7).items():
#     dictionary[word] += freq

# print size of dictionary
print(len(dictionary))
# print top 15 most frequent words
print("Biggest tokens:")
print(sorted(dictionary.items(), key=lambda x: x[1], reverse=True)[:15])

# print the dictionary first 100 words
print("First 100 words in the dictionary")
print(list(dictionary.items())[:100])
# print(dictionary)

49616
Top 15 most frequent words
[('marco', 20923), ('renames', 20922), ('vscode', 20921), ('sylva', 20920), ('materially', 20919), ('concealing', 20918), ('reasons—from', 20917), ('discusion', 20916), ('annihilated', 20915), ('zetachain', 20913), ('unplug', 20912), ('🚀🎱', 20910), ('stunner', 20908), ('grating', 20907), ('antic', 20903)]
First 100 words in the dictionary
[('sos', 1), ('eos', 2), ('ethereum', 3), ('bitcoin', 4), ('eth', 5), ('btc', 6), ('divi', 16109), ('project', 381), ('update', 521), ('september', 141), ('month', 43), ('review', 3723), ('great', 518), ('$', 7), ('guy', 315), ('keeping', 1125), ('busy', 11372), ('wait', 1161), ('mobile', 146), ('wallet', 231), ('heard', 990), ('going', 116), ('demo', 2474), ('vega', 14709), ('blockchain', 161), ('week', 254), ('wish', 1182), ('could', 480), ('go', 106), ('want', 249), ('world', 139), ('use', 220), ('crypto', 156), ('technology', 92), ('need', 344), ('easy', 1979), ('thank', 1307), ('god', 320), ('team', 159), ('got', 

In [81]:
# access a word token
print(dictionary['wreaks'])

5718


In [79]:
# print word with token 2295
for word, token in dictionary.items():
    if token == 5718:
        print(word)

sigh
wreaks
metatransaction


In [64]:
# save dictionary
import pickle
with open('dictionary.pickle', 'wb') as handle:
    pickle.dump(dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

# load dictionary
# with open('dictionary.pickle', 'rb') as handle:
#     dictionary = pickle.load(handle)